In [1]:
pip install azure-iot-device

     ---------------------------------------- 0.0/99.4 kB ? eta -:--:--
     ------- ------------------------------ 20.5/99.4 kB 640.0 kB/s eta 0:00:01
     ------------------------------------- -- 92.2/99.4 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 99.4/99.4 kB 948.2 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/168.5 kB ? eta -:--:--
   ---------------------------------------- 168.5/168.5 kB 9.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/64.9 kB ? eta -:--:--
   ---------------------------------------- 64.9/64.9 kB 3.4 MB/s eta 0:00:00
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=65648 sha256=beca0b2c8d6e781834d30cbd789711aff00d3d2106fba485473fbeeb063d93ee
  Stored in directory: c:\users\justi\appdata\local\pip\cache\wheels\23\d5\af\1f3cbcc350dec9d8e95040f388e0163d132eff0c9a453db659
Su

In [3]:
from azure.iot.device import IoTHubDeviceClient
import time
import json
import random

In [ ]:
from azure.iot.device import IoTHubDeviceClient
import time
import json
import random

# Replace this with the connection string of the IoT device you registered
CONNECTION_STRING = "HostName=MasonSquareIoTHub.azure-devices.net;DeviceId=spoof0001;SharedAccessKey=Xj1HrHB8dUE5ajQpjfRRFKTffX/JRoFqAm9wa/WmsEM="

# Create an IoT Hub client
client = IoTHubDeviceClient.create_from_connection_string(CONNECTION_STRING)

def send_data():
    try:
        room_numbers = [
            *range(1000, 1010),  # Floor 1, rooms 1000-1009
            *range(2000, 2010),  # Floor 2, rooms 2000-2009
            *range(3000, 3010),  # Floor 3, rooms 3000-3009
            *range(4000, 4010),  # Floor 4, rooms 4000-4009
            *range(5000, 5010)   # Floor 5, rooms 5000-5009
        ]

        for room_id in room_numbers:
            # Simulating the data to send with more realistic variations
            temperature = round(random.uniform(21.5, 24.0), 2)  # Typical office temperature range
            humidity = round(random.uniform(40, 60), 2)  # Moderate humidity level in an office
            oxygen = round(random.uniform(95, 99), 2)  # Oxygen level is usually stable
            co = round(random.uniform(0, 5), 2)  # Low carbon monoxide levels
            co2 = round(random.uniform(400, 800), 2)  # Typical indoor CO2 concentration
            energy_usage = round(random.uniform(150, 250), 2)  # Energy usage per room, moderate variation
            water_usage = round(random.uniform(5, 15), 2)  # Lower water usage for office spaces
            occupancy = random.randint(0, 40)  # Generate a whole number for occupancy
            
            # Creating the JSON message for each room
            message = {
                "deviceId": f"room_{room_id}",
                "timestamp": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()),
                "temperature": temperature,
                "humidity": humidity,
                "oxygen": oxygen,
                "carbon_monoxide": co,
                "carbon_dioxide": co2,
                "energy_usage": energy_usage,
                "water_usage": water_usage,
                "occupancy": occupancy,  # Occupancy as a whole number
            }
            
            # Convert message to JSON
            message_json = json.dumps(message)
            
            # Send message to IoT Hub
            print(f"Sending message for room {room_id}: {message_json}")
            client.send_message(message_json)
            
            # Sleep for a short time between room messages to avoid overwhelming the device
            time.sleep(0.5)

    except KeyboardInterrupt:
        print("IoT Hub client stopped.")

if __name__ == "__main__":
    while True:
        send_data()
        print("Waiting for 5 minutes before sending the next batch...")
        time.sleep(300)  # Wait for 5 minutes


Sending message for room 1000: {"deviceId": "room_1000", "timestamp": "2024-10-12T17:32:14Z", "temperature": 22.79, "humidity": 45.2, "oxygen": 97.49, "carbon_monoxide": 3.12, "carbon_dioxide": 512.99, "energy_usage": 205.62, "water_usage": 5.56, "occupancy": 11}
Sending message for room 1001: {"deviceId": "room_1001", "timestamp": "2024-10-12T17:32:15Z", "temperature": 21.82, "humidity": 52.93, "oxygen": 95.85, "carbon_monoxide": 0.03, "carbon_dioxide": 524.85, "energy_usage": 237.13, "water_usage": 14.42, "occupancy": 33}
Sending message for room 1002: {"deviceId": "room_1002", "timestamp": "2024-10-12T17:32:16Z", "temperature": 23.42, "humidity": 42.95, "oxygen": 98.63, "carbon_monoxide": 4.3, "carbon_dioxide": 643.51, "energy_usage": 242.03, "water_usage": 10.85, "occupancy": 35}
Sending message for room 1003: {"deviceId": "room_1003", "timestamp": "2024-10-12T17:32:16Z", "temperature": 23.4, "humidity": 46.23, "oxygen": 96.15, "carbon_monoxide": 3.03, "carbon_dioxide": 623.46, "en

In [21]:
import csv
import time
import random
from datetime import datetime, timedelta
from azure.iot.device import IoTHubDeviceClient

# Replace this with the connection string of the IoT device you registered
CONNECTION_STRING = "HostName=MasonSquareIoTHub.azure-devices.net;DeviceId=spoof0001;SharedAccessKey=Xj1HrHB8dUE5ajQpjfRRFKTffX/JRoFqAm9wa/WmsEM="

# Create an IoT Hub client
client = IoTHubDeviceClient.create_from_connection_string(CONNECTION_STRING)

def write_to_csv(data, filename='iot_data.csv'):
    file_exists = False
    try:
        file_exists = open(filename).read()  # Check if the file exists to determine if header is needed
    except FileNotFoundError:
        pass

    # Write data to CSV
    with open(filename, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=data.keys())
        if not file_exists:  # Write the header if file does not exist
            writer.writeheader()
        writer.writerow(data)

def send_data(total_records=6840, days=40):
    room_numbers = [
        *range(1000, 1010),  # Floor 1, rooms 1000-1009
        *range(2000, 2010),  # Floor 2, rooms 2000-2009
        *range(3000, 3010),  # Floor 3, rooms 3000-3009
        *range(4000, 4010),  # Floor 4, rooms 4000-4009
        *range(5000, 5010)   # Floor 5, rooms 5000-5009
    ]
    
    total_generated = 0
    records_per_room = total_records // len(room_numbers)  # Ensure each room generates an equal number of records

    # Simulated start time
    simulated_time = datetime(2024, 1, 1, 0, 0, 0)  # Start on January 1st at midnight

    while total_generated < total_records:
        for room_id in room_numbers:
            # If the required number of records has already been generated, stop
            if total_generated >= total_records:
                break

            # Simulating the data to send with more realistic variations
            temperature = round(random.uniform(21.5, 24.0), 2)  # Typical office temperature range
            humidity = round(random.uniform(40, 60), 2)  # Moderate humidity level in an office
            oxygen = round(random.uniform(95, 99), 2)  # Oxygen level is usually stable
            co = round(random.uniform(0, 5), 2)  # Low carbon monoxide levels
            co2 = round(random.uniform(400, 800), 2)  # Typical indoor CO2 concentration
            energy_usage = round(random.uniform(150, 250), 2)  # Energy usage per room, moderate variation
            water_usage = round(random.uniform(5, 15), 2)  # Lower water usage for office spaces
            occupancy = random.randint(0, 40)  # Generate a whole number for occupancy

            # Creating the message dictionary
            message = {
                "deviceId": f"room_{room_id}",
                "timestamp": simulated_time.strftime("%Y-%m-%dT%H:%M:%SZ"),  # Use the spoofed time
                "temperature": temperature,
                "humidity": humidity,
                "oxygen": oxygen,
                "carbon_monoxide": co,
                "carbon_dioxide": co2,
                "energy_usage": energy_usage,
                "water_usage": water_usage,
                "occupancy": occupancy,  # Occupancy as a whole number
            }

            # Write data to CSV
            write_to_csv(message)

            # Increment the row count
            total_generated += 1

        # After one full loop (all rooms), increment the time by 5 minutes
        simulated_time += timedelta(minutes=5)

        day = 0
        
        # After 24 hours (1440 minutes or 288 iterations of 5-minute increments), reset to the next day
        if simulated_time.hour == 0 and simulated_time.minute == 0:
            simulated_time = simulated_time.replace(hour=0, minute=0)
            day += 1
            print(day)

if __name__ == "__main__":
    send_data(6840)
